<a href="https://colab.research.google.com/github/Anuragjha0936/Machine-learning-/blob/main/Prediction_test/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
ticker = "BTC-USD"

data = yf.download(
    ticker,
    start="2016-01-01",
    end="2024-01-01",
    interval="1d"
)


/tmp/ipython-input-346863845.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


In [3]:
# Log returns (better for volatile assets)
data["log_return"] = np.log(data["Close"] / data["Close"].shift(1))

# Short + medium moving averages
data["ma_5"] = data["Close"].rolling(5).mean()
data["ma_10"] = data["Close"].rolling(10).mean()
data["ma_20"] = data["Close"].rolling(20).mean()

# Volatility (short-term)
data["volatility_5"] = data["log_return"].rolling(5).std()
data["volatility_10"] = data["log_return"].rolling(10).std()

# Momentum
data["momentum_5"] = data["Close"] - data["Close"].shift(5)
data["momentum_10"] = data["Close"] - data["Close"].shift(10)


In [4]:
# 1 = price goes up tomorrow, 0 = goes down
data["target"] = (data["Close"].shift(-1) > data["Close"]).astype(int)

data = data.dropna()


In [5]:
features = [
    "log_return",
    "ma_5",
    "ma_10",
    "ma_20",
    "volatility_5",
    "volatility_10",
    "momentum_5",
    "momentum_10"
]

X = data[features]
y = data["target"]


In [6]:
split = int(len(data) * 0.8)

X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]


In [7]:
model = XGBClassifier(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.03,
    subsample=0.7,
    colsample_bytree=0.7,
    gamma=0.2,
    reg_alpha=0.5,
    reg_lambda=1.5,
    eval_metric="logloss",
    random_state=42
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0.2,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, ...)

In [8]:
preds = model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, preds))

print("\nClassification Report:")
print(classification_report(y_test, preds))


Confusion Matrix:
[[ 55 249]
 [ 49 228]]

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.18      0.27       304
           1       0.48      0.82      0.60       277

    accuracy                           0.49       581
   macro avg       0.50      0.50      0.44       581
weighted avg       0.50      0.49      0.43       581



In [9]:
probs = model.predict_proba(X_test)[:, 1]

signals = []
for p in probs:
    if p > 0.57:
        signals.append("BUY")
    else:
        signals.append("SELL")

print(signals[:10])


['SELL', 'BUY', 'SELL', 'SELL', 'BUY', 'BUY', 'BUY', 'SELL', 'SELL', 'BUY']
